In [1]:
from utils import init_experiment
model_name = "bert-base-uncased"
init_experiment({"model": model_name})

wandb: Currently logged in as: kydlicek-hynek (hynky). Use `wandb login --relogin` to force relogin


In [182]:
import pandas as pd
train = pd.read_parquet("../artifacts/feature_store/yelp2020/train/")
test = pd.read_parquet("../artifacts/feature_store/yelp2020/test/")



In [185]:
# we will use small sample of the data 200k
train = train.sample(800000)
# split into train and validation
train = train.sample(frac=0.95, random_state=42)
val = train.drop(train.index)

In [208]:
from typing import List
from transformers import PreTrainedTokenizer
from datasets import Dataset


class YelpDataset(torch.utils.data.Dataset):
    def tokenize(self, text: List[str], tokenizer: PreTrainedTokenizer):
        return self.tokenizer(text, truncation=True, padding=True)

    def prepare_df(self, df: pd.DataFrame):
        dataset = Dataset.from_pandas(df)
        dataset = dataset.class_encode_column("usefulness")
        dataset = dataset.rename_column("usefulness", "labels")
        return dataset


    def __init__(self, data: pd.DataFrame, tokenizer: PreTrainedTokenizer):
        self.tokenizer = tokenizer
        dataset = self.prepare_df(data)
        self.tokenized_text = self.tokenize(dataset["text"], tokenizer)
        self.labels = dataset["labels"]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.tokenized_text[idx], self.labels[idx]

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_yelp_dataset = YelpDataset(train, tokenizer)
val_yelp_dataset = YelpDataset(val, tokenizer)
test_yelp_dataset = YelpDataset(test, tokenizer)

NameError: name 'YelpDataset' is not defined

In [175]:
transformer = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [179]:
from transformers import TrainingArguments
from transformers import Trainer
from utils import get_result

training_args = TrainingArguments(
    output_dir="../artifacts/models/yelp2020",
    num_train_epochs=1,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    logging_steps=10,
    report_to="wandb",
)



compute_metrics = lambda pred: {"f1_macro": get_result(pred.label_ids, pred.predictions.argmax(-1)).metric_value}
trainer = Trainer(
    model=transformer,
    args=training_args,
    train_dataset=train_yelp_dataset,
    eval_dataset=val_yelp_dataset,
    compute_metrics=compute_metrics,
)

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
trainer.train()